In [14]:
import pandas as pd
import numpy as np
import os
import re
import ast
from collections import Counter

from konlpy.tag import Okt  
from ckonlpy.tag import Twitter, Postprocessor
from tensorflow.keras.preprocessing.text import Tokenizer
import sys
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 로딩

In [2]:
df = pd.read_excel('crawlingdata.xlsx')

In [4]:
df.drop(df.columns[0],axis=1,inplace=True)
df.head()

,검색어,날짜,언론사,제목,요약,링크
0,메리츠화재,3일 전,머니S,"김용범 메리츠화재 대표, '영업통' 김경환 부사장 앉혔다… 이유는?",김용범 메리츠화재 대표가 김경환 전무를 부사장으로 승진시키는 등 지난 23일 임원인...,http://moneys.mt.co.kr/news/mwView.php?no=2022...
1,메리츠화재,4일 전,매경이코노미,내년 車 보험료 내린다…현대·KB·메리츠화재 등 줄 인하,"롯데손보 2.9%로 인하폭 가장 커 메리츠 2.5%, 현대·KB 2.0% 등 현대해...",https://www.mk.co.kr/article/10578542
2,메리츠화재,3일 전,팍스넷뉴스,"메리츠證, 장원재 사장 승진…""리스크관리 전문가""","메리츠화재 최고리스크관리책임자(CRO), 메리츠금융지주 CRO 등을 역임했다. 메리...",https://www.paxnetnews.com/articles/96200
3,메리츠화재,3일 전,뉴스핌,"메리츠금융그룹, 정기 임원 인사 실시",[서울=뉴스핌] 김승동 기자 = 메리츠화재 본사 사옥 2020.09.03 0I087...,http://www.newspim.com/news/view/20221223000452
4,메리츠화재,3일 전,프라임경제,[인사] 메리츠화재,메리츠화재(000060)는 2023년 1월1일자로 임원 승진 인사를 단행했다고 23...,http://www.newsprime.co.kr/news/article.html?n...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426837 entries, 0 to 426836
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   검색어     426837 non-null  object
 1   날짜      426837 non-null  object
 2   언론사     426837 non-null  object
 3   제목      426837 non-null  object
 4   요약      426656 non-null  object
 5   링크      65526 non-null   object
dtypes: object(6)
memory usage: 19.5+ MB


# 정제

In [8]:
# 공백 제거    
pattern = '[\r|\n|\t]'
df['제목'] = df['제목'].apply(lambda x: re.sub(pattern=pattern, repl=' ', string=str(x)))
df['요약'] = df['요약'].apply(lambda x: re.sub(pattern=pattern, repl=' ', string=str(x)))

# 특수기호 제거    
pattern = '[^\w|\s]'
df['제목'] = df['제목'].apply(lambda x: re.sub(pattern=pattern, repl=' ', string=str(x)))
df['요약'] = df['요약'].apply(lambda x: re.sub(pattern=pattern, repl=' ', string=str(x)))

# 텍스트 양옆 공백 제거    
df['제목'] = df['제목'].apply(lambda x: x.strip())
df['요약'] = df['요약'].apply(lambda x: x.strip())

# 영어 대문자 통일    
df['제목'] = df['제목'].apply(lambda x: x.upper())
df['요약'] = df['요약'].apply(lambda x: x.upper())

In [9]:
df.head()

,검색어,날짜,언론사,제목,요약,링크
0,메리츠화재,3일 전,머니S,김용범 메리츠화재 대표 영업통 김경환 부사장 앉혔다 이유는,김용범 메리츠화재 대표가 김경환 전무를 부사장으로 승진시키는 등 지난 23일 임원인...,http://moneys.mt.co.kr/news/mwView.php?no=2022...
1,메리츠화재,4일 전,매경이코노미,내년 車 보험료 내린다 현대 KB 메리츠화재 등 줄 인하,롯데손보 2 9 로 인하폭 가장 커 메리츠 2 5 현대 KB 2 0 등 현대해...,https://www.mk.co.kr/article/10578542
2,메리츠화재,3일 전,팍스넷뉴스,메리츠證 장원재 사장 승진 리스크관리 전문가,메리츠화재 최고리스크관리책임자 CRO 메리츠금융지주 CRO 등을 역임했다 메리...,https://www.paxnetnews.com/articles/96200
3,메리츠화재,3일 전,뉴스핌,메리츠금융그룹 정기 임원 인사 실시,서울 뉴스핌 김승동 기자 메리츠화재 본사 사옥 2020 09 03 0I0870...,http://www.newspim.com/news/view/20221223000452
4,메리츠화재,3일 전,프라임경제,인사 메리츠화재,메리츠화재 000060 는 2023년 1월1일자로 임원 승진 인사를 단행했다고 23...,http://www.newsprime.co.kr/news/article.html?n...


# 토큰화

In [10]:
word_lst = ['계약','체결','합병','인수','흡수','MOU','합작','협력','협약','협정','파트너쉽']
words = '|'.join(word_lst)
df_join = df[(df['요약'].str.contains(words))&~(df['요약'].str.contains('계약학과'))]
df_join.head()

,검색어,날짜,언론사,제목,요약,링크
8,메리츠화재,5일 전,뉴스1,메리츠화재 내년 개인용 자동차보험료 2 5 인하 결정,메리츠화재는 21일 개인용 자동차보험료를 내년 2월27일 이후 책임 개시 계약부터 ...,https://www.news1.kr/articles/4901625
13,메리츠화재,4일 전,스마트경제,메리츠화재 내년 자동차보험료 2 5 인하 예정,메리츠화재 서울 강남 본사 전경 사진 메리츠화재 메리츠화재가 개인용 자동차보험...,http://www.dailysmart.co.kr/news/articleView.h...
18,메리츠화재,5일 전,SBS Biz,롯데손보 메리츠에 이어 KB손보도 車보험료 2 대 인하,롯데손해보험과 메리츠화재에 이어 KB손해보험도 내년도 개인용 자동차보험료 2 대 인...,https://biz.sbs.co.kr/article_hub/20000094730?...
23,메리츠화재,5일 전,이데일리,메리츠화재 내년 1월 5일 주주총회 소집,메리츠화재 000060 는 2023년 1월 5일 서울시 강남구 메리츠타워에서 주주총...,http://www.edaily.co.kr/news/newspath.asp?news...
30,메리츠화재,2022.12.18.,충남일보,대전 서구 메리츠화재 걱정해결사업 협약 체결 저소득 주민 돕는다,대전 서구와 메리츠화재는 지난 16일 저소득 주민의 위기 해소와 자립 도모를 위한...,http://www.chungnamilbo.co.kr/news/articleView...


In [12]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48653 entries, 8 to 426822
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   검색어     48653 non-null  object
 1   날짜      48653 non-null  object
 2   언론사     48653 non-null  object
 3   제목      48653 non-null  object
 4   요약      48653 non-null  object
 5   링크      8421 non-null   object
dtypes: object(6)
memory usage: 2.6+ MB


In [15]:
okt=Okt() 

df_join = df_join.dropna(axis=0, subset='요약')
df_dict = pd.read_excel('상장목록.xlsx')


# 하나의 단어로 인식할 단어 지정
def add_words():
    sys.path.insert(0, '../')
    twitter = Twitter(use_twitter_dictionary=False)
    twitter.add_dictionary(list(df_dict.iloc[:,0].value_counts().index), 'Noun')
    return twitter

# 토큰화
def make_token(x):
    word_token = add_words().morphs(x)
    result = [w for w in word_token]
    return result

# 품사 태깅
def pos(x):
    word_token = add_words().pos(x)
    result = [w for w in word_token]
    return result


df_join['token'] = df_join['요약'].apply(lambda x: make_token(x))
df_join['pos'] = df_join['요약'].apply(lambda x: pos(x))
df_join['noun'] = df_join['pos'].apply(lambda x: [w[0] for w in x if w[1]=='Noun'])

df_join.to_csv('뉴스크롤링_company_tokenized.csv',index=False,encoding='utf-8-sig')

In [16]:
df_join

,검색어,날짜,언론사,제목,요약,링크,token,pos,noun
8,메리츠화재,5일 전,뉴스1,메리츠화재 내년 개인용 자동차보험료 2 5 인하 결정,메리츠화재는 21일 개인용 자동차보험료를 내년 2월27일 이후 책임 개시 계약부터 ...,https://www.news1.kr/articles/4901625,"[메리츠화재, 는, 21일, 개인, 용, 자동차, 보험료, 를, 내년, 2월, 27...","[(메리츠화재, Noun), (는, Verb), (21일, Number), (개인,...","[메리츠화재, 개인, 용, 자동차, 보험료, 내년, 이후, 책임, 개시, 계약, 인..."
13,메리츠화재,4일 전,스마트경제,메리츠화재 내년 자동차보험료 2 5 인하 예정,메리츠화재 서울 강남 본사 전경 사진 메리츠화재 메리츠화재가 개인용 자동차보험...,http://www.dailysmart.co.kr/news/articleView.h...,"[메리츠화재, 서울, 강남, 본사, 전경, 사진, 메리츠화재, 메리츠화재, 가, 개...","[(메리츠화재, Noun), (서울, Noun), (강남, Noun), (본사, N...","[메리츠화재, 서울, 강남, 본사, 전경, 사진, 메리츠화재, 메리츠화재, 개인, ..."
18,메리츠화재,5일 전,SBS Biz,롯데손보 메리츠에 이어 KB손보도 車보험료 2 대 인하,롯데손해보험과 메리츠화재에 이어 KB손해보험도 내년도 개인용 자동차보험료 2 대 인...,https://biz.sbs.co.kr/article_hub/20000094730?...,"[롯데손해보험, 과, 메리츠화재, 에, 이어, KB, 손해, 보험, 도, 내년, 도...","[(롯데손해보험, Noun), (과, Noun), (메리츠화재, Noun), (에,...","[롯데손해보험, 과, 메리츠화재, 손해, 보험, 내년, 개인, 용, 자동차, 보험료..."
23,메리츠화재,5일 전,이데일리,메리츠화재 내년 1월 5일 주주총회 소집,메리츠화재 000060 는 2023년 1월 5일 서울시 강남구 메리츠타워에서 주주총...,http://www.edaily.co.kr/news/newspath.asp?news...,"[메리츠화재, 000060, 는, 2023년, 1월, 5일, 서울시, 강남구, 메,...","[(메리츠화재, Noun), (000060, Number), (는, Verb), (...","[메리츠화재, 서울시, 강남구, 메, 리츠, 타워, 주주총회, 소집, 공시, 회의,..."
30,메리츠화재,2022.12.18.,충남일보,대전 서구 메리츠화재 걱정해결사업 협약 체결 저소득 주민 돕는다,대전 서구와 메리츠화재는 지난 16일 저소득 주민의 위기 해소와 자립 도모를 위한...,http://www.chungnamilbo.co.kr/news/articleView...,"[대전, 서구, 와, 메리츠화재, 는, 지난, 16일, 저, 소득, 주민, 의, 위...","[(대전, Noun), (서구, Noun), (와, Josa), (메리츠화재, No...","[대전, 서구, 메리츠화재, 지난, 소득, 주민, 위기, 해소, 자립, 도모, 위,..."
...,...,...,...,...,...,...,...,...,...
426811,SK스퀘어,15면 1단,디지털타임스,SK하이닉스 솔리다임과 통합 시너지 극대화,박 부회장은 앞서 지난 28일 SK하이닉스 모회사인 SK스퀘어 주주총회에서 영국 반...,NaN,"[박, 부회장, 은, 앞서, 지난, 28일, SK하이닉스, 모회사, 인, SK스퀘어...","[(박, Noun), (부회장, Noun), (은, Josa), (앞서, Noun)...","[박, 부회장, 앞서, 지난, SK하이닉스, 모회사, SK스퀘어, 주주총회, 영국,..."
426813,SK스퀘어,2022.03.31.,베타뉴스,SK하이닉스 ARM 지분 인수 검토 미래 사업 성장성 확보 나서나,지난 28일에도 박 부회장은 SK하이닉스의 모기업 SK스퀘어 주총에서도 ARM 인수...,NaN,"[지난, 28일, 에도, 박, 부회장, 은, SK하이닉스, 의, 모, 기업, SK스...","[(지난, Noun), (28일, Number), (에도, Foreign), (박,...","[지난, 박, 부회장, SK하이닉스, 의, 기업, SK스퀘어, 총, 인수, 계획, ..."
426814,SK스퀘어,2022.03.30.,SBS Biz,SK 박정호 M A 추진 에디슨 강영권 손해배상 청구,박 부회장은 이틀 전 SK스퀘어 주주총회에서도 다음 달 실리콘밸리에서 협의를 이어...,NaN,"[박, 부회장, 은, 이틀, 전, SK스퀘어, 주주총회, 에서도, 다음, 달, 실리...","[(박, Noun), (부회장, Noun), (은, Josa), (이틀, Noun)...","[박, 부회장, 이틀, 전, SK스퀘어, 주주총회, 다음, 달, 실리콘밸리, 협의,..."
426818,SK스퀘어,2022.03.30.,아시아투데이,SK하이닉스 인텔 키파운드리 이어 ARM에 관심 사업 다각화 속도,박 부회장은 지난 28일 SK스퀘어 주주총회에서도 반도체 업체는 규모가 큰 곳부터...,NaN,"[박, 부회장, 은, 지난, 28일, SK스퀘어, 주주총회, 에서도, 반도체, 업체...","[(박, Noun), (부회장, Noun), (은, Josa), (지난, Noun)...","[박, 부회장, 지난, SK스퀘어, 주주총회, 반도체, 업체, 규모, 곳, 곳, 계..."


# 기업명 분리

In [17]:
df_join2 = df_join.copy()

In [18]:
dic = pd.read_excel('상장목록.xlsx')
company_list = list(dic['기업명'])
company_list

['3S',
 '가비아',
 '가온미디어',
 '가온전선',
 '가온칩스',
 '감성코퍼레이션',
 '강남제비스코',
 '강스템바이오텍',
 '강원랜드',
 '강원에너지',
 '갤럭시아머니트리',
 '갤럭시아에스엠',
 '경남스틸',
 '경남제약',
 '경농',
 '경동나비엔',
 '경동도시가스',
 '경동인베스트',
 '경동제약',
 '경방',
 '경보제약',
 '경인양행',
 '경인전자',
 '경창산업',
 '계룡건설산업',
 '계양전기',
 '고려산업',
 '고려시멘트',
 '고려신용정보',
 '고려아연',
 '고려제강',
 '고려제약',
 '고바이오랩',
 '고영',
 '골드앤에스',
 '골드퍼시픽',
 '골든센츄리',
 '골프존',
 '골프존뉴딘홀딩스',
 '골프존데카',
 '공구우먼',
 '관악산업',
 '광동제약',
 '광림',
 '광명전기',
 '광무',
 '광전자',
 '광주신세계',
 '광진실업',
 '교보증권',
 '교촌에프앤비',
 '구영테크',
 '국도화학',
 '국동',
 '국보',
 '국보디자인',
 '국순당',
 '국영지앤엠',
 '국일신동',
 '국일제지',
 '국전약품',
 '국제약품',
 '굿센',
 '그래디언트',
 '그리티',
 '그린케미칼',
 '그린플러스',
 '극동유화',
 '글로벌에스엠',
 '글로벌텍스프리',
 '글로본',
 '금강공업',
 '금강철강',
 '금비',
 '금양',
 '금호건설',
 '금호석유화학',
 '금호에이치티',
 '금호전기',
 '금호타이어',
 '금화피에스시',
 '기가레인',
 '기산텔레콤',
 '기신정기',
 '기아',
 '기업은행',
 '까뮤이앤씨',
 '까스텔바작',
 '깨끗한나라',
 '나노',
 '나노브릭',
 '나노신소재',
 '나노씨엠에스',
 '나노엔텍',
 '나노캠텍',
 '나눔테크',
 '나라소프트',
 '나라엠앤디',
 '나래나노텍',
 '나무가',
 '나무기술',
 '나스미디어',
 '나우코스',
 '나우IB',
 '나이벡',
 '나이스디앤비',
 '나이스정보통신',

In [19]:
def func(x):
    lst = []
    for n in x:
        if n in company_list:
            lst.append(n)
    return lst

df_join2['company'] = df_join2['noun'].apply(lambda x:func(x))
df_join2.reset_index(drop=True,inplace=True)

In [20]:
df_join2

,검색어,날짜,언론사,제목,요약,링크,token,pos,noun,company
0,메리츠화재,5일 전,뉴스1,메리츠화재 내년 개인용 자동차보험료 2 5 인하 결정,메리츠화재는 21일 개인용 자동차보험료를 내년 2월27일 이후 책임 개시 계약부터 ...,https://www.news1.kr/articles/4901625,"[메리츠화재, 는, 21일, 개인, 용, 자동차, 보험료, 를, 내년, 2월, 27...","[(메리츠화재, Noun), (는, Verb), (21일, Number), (개인,...","[메리츠화재, 개인, 용, 자동차, 보험료, 내년, 이후, 책임, 개시, 계약, 인...",[메리츠화재]
1,메리츠화재,4일 전,스마트경제,메리츠화재 내년 자동차보험료 2 5 인하 예정,메리츠화재 서울 강남 본사 전경 사진 메리츠화재 메리츠화재가 개인용 자동차보험...,http://www.dailysmart.co.kr/news/articleView.h...,"[메리츠화재, 서울, 강남, 본사, 전경, 사진, 메리츠화재, 메리츠화재, 가, 개...","[(메리츠화재, Noun), (서울, Noun), (강남, Noun), (본사, N...","[메리츠화재, 서울, 강남, 본사, 전경, 사진, 메리츠화재, 메리츠화재, 개인, ...","[메리츠화재, 메리츠화재, 메리츠화재]"
2,메리츠화재,5일 전,SBS Biz,롯데손보 메리츠에 이어 KB손보도 車보험료 2 대 인하,롯데손해보험과 메리츠화재에 이어 KB손해보험도 내년도 개인용 자동차보험료 2 대 인...,https://biz.sbs.co.kr/article_hub/20000094730?...,"[롯데손해보험, 과, 메리츠화재, 에, 이어, KB, 손해, 보험, 도, 내년, 도...","[(롯데손해보험, Noun), (과, Noun), (메리츠화재, Noun), (에,...","[롯데손해보험, 과, 메리츠화재, 손해, 보험, 내년, 개인, 용, 자동차, 보험료...","[롯데손해보험, 메리츠화재]"
3,메리츠화재,5일 전,이데일리,메리츠화재 내년 1월 5일 주주총회 소집,메리츠화재 000060 는 2023년 1월 5일 서울시 강남구 메리츠타워에서 주주총...,http://www.edaily.co.kr/news/newspath.asp?news...,"[메리츠화재, 000060, 는, 2023년, 1월, 5일, 서울시, 강남구, 메,...","[(메리츠화재, Noun), (000060, Number), (는, Verb), (...","[메리츠화재, 서울시, 강남구, 메, 리츠, 타워, 주주총회, 소집, 공시, 회의,...","[메리츠화재, 메리츠금융지주]"
4,메리츠화재,2022.12.18.,충남일보,대전 서구 메리츠화재 걱정해결사업 협약 체결 저소득 주민 돕는다,대전 서구와 메리츠화재는 지난 16일 저소득 주민의 위기 해소와 자립 도모를 위한...,http://www.chungnamilbo.co.kr/news/articleView...,"[대전, 서구, 와, 메리츠화재, 는, 지난, 16일, 저, 소득, 주민, 의, 위...","[(대전, Noun), (서구, Noun), (와, Josa), (메리츠화재, No...","[대전, 서구, 메리츠화재, 지난, 소득, 주민, 위기, 해소, 자립, 도모, 위,...","[메리츠화재, 메리츠화재]"
...,...,...,...,...,...,...,...,...,...,...
48648,SK스퀘어,15면 1단,디지털타임스,SK하이닉스 솔리다임과 통합 시너지 극대화,박 부회장은 앞서 지난 28일 SK하이닉스 모회사인 SK스퀘어 주주총회에서 영국 반...,NaN,"[박, 부회장, 은, 앞서, 지난, 28일, SK하이닉스, 모회사, 인, SK스퀘어...","[(박, Noun), (부회장, Noun), (은, Josa), (앞서, Noun)...","[박, 부회장, 앞서, 지난, SK하이닉스, 모회사, SK스퀘어, 주주총회, 영국,...","[SK하이닉스, SK스퀘어]"
48649,SK스퀘어,2022.03.31.,베타뉴스,SK하이닉스 ARM 지분 인수 검토 미래 사업 성장성 확보 나서나,지난 28일에도 박 부회장은 SK하이닉스의 모기업 SK스퀘어 주총에서도 ARM 인수...,NaN,"[지난, 28일, 에도, 박, 부회장, 은, SK하이닉스, 의, 모, 기업, SK스...","[(지난, Noun), (28일, Number), (에도, Foreign), (박,...","[지난, 박, 부회장, SK하이닉스, 의, 기업, SK스퀘어, 총, 인수, 계획, ...","[SK하이닉스, SK스퀘어]"
48650,SK스퀘어,2022.03.30.,SBS Biz,SK 박정호 M A 추진 에디슨 강영권 손해배상 청구,박 부회장은 이틀 전 SK스퀘어 주주총회에서도 다음 달 실리콘밸리에서 협의를 이어...,NaN,"[박, 부회장, 은, 이틀, 전, SK스퀘어, 주주총회, 에서도, 다음, 달, 실리...","[(박, Noun), (부회장, Noun), (은, Josa), (이틀, Noun)...","[박, 부회장, 이틀, 전, SK스퀘어, 주주총회, 다음, 달, 실리콘밸리, 협의,...",[SK스퀘어]
48651,SK스퀘어,2022.03.30.,아시아투데이,SK하이닉스 인텔 키파운드리 이어 ARM에 관심 사업 다각화 속도,박 부회장은 지난 28일 SK스퀘어 주주총회에서도 반도체 업체는 규모가 큰 곳부터...,NaN,"[박, 부회장, 은, 지난, 28일, SK스퀘어, 주주총회, 에서도, 반도체, 업체...","[(박, Noun), (부회장, Noun), (은, Josa), (지난, Noun)...","[박, 부회장, 지난, SK스퀘어, 주주총회, 반도체, 업체, 규모, 곳, 곳, 계...","[SK스퀘어, 대상]"


# 기업 count - 코스피200기업만 

```kospi_counts``` 폴더 안에 저장

In [29]:
# 코스피200기업
kospi_lst = list(df_join2['검색어'].value_counts().index)
kospi_lst

['한화',
 '대우조선해양',
 '일진머티리얼즈',
 'KG스틸',
 'KT',
 'KT&G',
 '동원산업',
 '포스코케미칼',
 '한화시스템',
 '한국조선해양',
 '한화에어로스페이스',
 '롯데케미칼',
 '롯데정밀화학',
 '현대중공업',
 '삼성중공업',
 'SK',
 '현대차',
 'LG화학',
 '에스디바이오센서',
 '포스코인터내셔널',
 'LG에너지솔루션',
 'LIG넥스원',
 'LG',
 '아시아나항공',
 '삼성전자',
 '삼성물산',
 '한국항공우주',
 '두산퓨얼셀',
 '고려아연',
 '지누스',
 '한전KPS',
 '롯데지주',
 '한국전력',
 '한전기술',
 '현대미포조선',
 '대한항공',
 'DB손해보험',
 '두산에너빌리티',
 '한국가스공사',
 '에스엘',
 '한국콜마',
 'BGF리테일',
 '한미사이언스',
 'CJ',
 '현대글로비스',
 '두산',
 'SK스퀘어',
 '현대로템',
 '동서',
 'SKC',
 'GS',
 'SK이노베이션',
 '삼성엔지니어링',
 '기아',
 'LX인터내셔널',
 'DL',
 'LS',
 'SK네트웍스',
 'SK텔레콤',
 'OCI',
 '삼성SDI',
 'PI첨단소재',
 '씨에스윈드',
 'DL이앤씨',
 '현대제철',
 '현대위아',
 'HDC현대산업개발',
 'GS건설',
 '대우건설',
 '한진칼',
 '한샘',
 'GS리테일',
 '기업은행',
 '한국앤컴퍼니',
 '현대건설',
 '유한양행',
 '삼성바이오로직스',
 'SK바이오팜',
 '강원랜드',
 'LG유플러스',
 '녹십자홀딩스',
 '현대엘리베이터',
 '신한지주',
 '롯데제과',
 '영풍',
 'S-Oil',
 'KB금융',
 'LG전자',
 '신세계',
 '하나투어',
 '한화솔루션',
 'CJ대한통운',
 '한화생명',
 '한섬',
 '현대두산인프라코어',
 '오리온',
 '효성첨단소재',
 '대한전선',
 '현대홈쇼핑',
 '대웅',
 '코스맥스',
 '금호석유',
 'SK케미칼',
 '코웨

In [32]:
for kospi in kospi_lst:
    
    df_tmp = df_join2[df_join2['검색어']==kospi]
    df_tmp.reset_index(drop=True,inplace=True)
    
    lst = []
    df2 = df_tmp["company"]

    for i in range(len(df2)):
        lst.extend(df2[i])
        
    while kospi in lst: #자사 제거
        lst.remove(kospi)

    # 중복 단어 갯수 세기 
    counter = Counter(lst)
    df_count = pd.DataFrame(counter.most_common(),columns=['word','count'])
    
    df_count2 = df_count[df_count['word'].isin(kospi_lst)]
    
    df_count2.to_csv('./kospi_counts/{}.csv'.format(kospi),index=False,encoding='utf-8-sig')

# 기업 count - 코스피200+추가 100개 기업

## 전체 기업 count

In [33]:
# 코스피200이 아닌 기업 중 언급량 count
lst = []
df2 = df_join2["company"]

for i in range(len(df2)):
    lst.extend(df2[i])

for kospi in kospi_lst:
    while kospi in lst: 
        lst.remove(kospi)

# 중복 단어 갯수 세기 
counter = Counter(lst)
df_count = pd.DataFrame(counter.most_common(),columns=['word','count'])

In [43]:
# 상위 100개
nokospi_lst = list(df_count[0:100]['word'])

In [46]:
print(nokospi_lst)

['SG', '레이', '현대자동차', '태양', '국동', 'DB', '금호석유화학', '쌍용자동차', 'NEW', '한국전력공사', 'KD', '코오롱인더', '연우', '성일하이텍', '영풍제지', '효성중공업', '롯데칠성음료', 'KG케미칼', 'CS', '쌍방울', '선진', '현대리바트', '에스엘바이오닉스', 'STX중공업', '디오', 'E1', '카페24', 'KG이니시스', '테스', '코오롱', '현대건설기계', '한국콜마홀딩스', 'SK가스', '한진', '뉴트리', 'KG모빌리언스', 'HSD엔진', '오텍', '드래곤플라이', '진원생명과학', '흥국', 'CJ프레시웨이', '부광약품', '금호건설', '녹십자웰빙', '윈스', '대양금속', 'HDC', '우성', '종근당바이오', '에스트래픽', 'DL건설', 'ISC', '동양', '코리안리', 'KT서브마린', '전방', 'EG', '남해화학', '미래산업', '쏘카', '승일', '한화투자증권', '효성화학', '아스트', 'HK이노엔', '유안타증권', '진에어', 'SK증권', '한국철강', '솔루스첨단소재', '대동', '디아이', '넵튠', '광림', '엘앤에프', '컴투스', '현대일렉트릭', 'BGF', '롯데렌탈', '태광', '디에이테크놀로지', '하림', '에코프로에이치엔', '휴젤', '진도', '조비', '더존비즈온', '딜리', 'LF', '대덕', '일진홀딩스', 'LX하우시스', '대한제강', '큐브엔터', '깨끗한나라', '삼강엠앤티', '조일알미늄', '신성이엔지', '웅진']


## 추가 100개 기업 포함해서 count

```kospi_counts(2)``` 폴더 안에 저장

In [44]:
for kospi in kospi_lst:
    
    df_tmp = df_join2[df_join2['검색어']==kospi]
    df_tmp.reset_index(drop=True,inplace=True)
    
    lst = []
    df2 = df_tmp["company"]

    for i in range(len(df2)):
        lst.extend(df2[i])
        
    while kospi in lst: #자사 제거
        lst.remove(kospi)

    # 중복 단어 갯수 세기 
    counter = Counter(lst)
    df_count = pd.DataFrame(counter.most_common(),columns=['word','count'])
    
    df_count2 = df_count[(df_count['word'].isin(kospi_lst))|(df_count['word'].isin(nokospi_lst))]
    
    df_count2.to_csv('./kospi_counts(2)/{}.csv'.format(kospi),index=False,encoding='utf-8-sig')